In [1]:
import pandas as pd
import glob
import os
import xml.etree.ElementTree as ET
import re

# Load Data

In [2]:
# Full bvb export

path = 'data/csv/b3kat_export/'
files = glob.glob(os.path.join(path, '*.csv'))

li = []

for filename in files: 
    frame = pd.read_csv(filename, header=0)
    li.append(frame)

df_full = pd.concat(li, axis=0, ignore_index=True)

# No need for all the columns

df_full= df_full[['id', 'lang', 'authors', 'parentId']]

C:\Users\hanna\AppData\Local\Temp/ipykernel_3328/2627611449.py:9: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(filename, header=0)


In [15]:
# signatures

df_sig = pd.read_csv('data/csv/sig_updated.csv')

df_sig = df_sig[['sys', 'text', 'text_3', 'text_2', 'text_1']]

C:\Users\hanna\AppData\Local\Temp/ipykernel_3328/764943422.py:3: DtypeWarning: Columns (2,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sig = pd.read_csv('data/csv/sig_updated.csv')


In [16]:
# Books available in the library

df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0, 
                          names = ['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr', 'inventory_date'])

C:\Users\hanna\AppData\Local\Temp/ipykernel_3328/3202955160.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0,


In [17]:
print(df_freihand.columns)
print(df_full.columns)
print(df_sig.columns)

Index(['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr',
       'inventory_date'],
      dtype='object')
Index(['id', 'lang', 'authors', 'parentId'], dtype='object')
Index(['sys', 'text', 'text_3', 'text_2', 'text_1'], dtype='object')


In [19]:
df_sig = df_sig.fillna('')
df_freihand = df_freihand.fillna('')


### Merging Signatures and Books

In [22]:
# Extracting the known signature types

patterns = [
    r'^([a-zA-Z]{1,2}) (\d+)',
    r'^[CWZ][amopsu]-[A-Z]{3}\s\d{1,4}',
    r'^[EXY][a-z]?-[A-Z]{3}\s\d{1,4}'
]

def clean_signature(signature):
    match = None
    for pattern in patterns:
        match = re.search(pattern, signature)
        if match:
            break
    
    return match.group() if match else None

df_freihand['signature_clean'] = df_freihand['signature'].apply(clean_signature)

# Merging on the signatures

merged = pd.merge(df_freihand, df_sig, left_on='signature_clean', right_on='sys', how= 'inner', indicator=True)
both = merged.loc[merged['_merge'] == 'both']

# percentage of documents with recognized signatures

rec_sigs = len(both) / len(df_freihand) * 100
print(f'Books in Freihand matched with signature: {rec_sigs:.1f} %')

# signatures not in the list marked with left only

left_only = merged.loc[merged['_merge'] == 'left_only']

# Keeping this to check which ones are still missing

left_only.groupby(left_only.signature[:6]).size().to_csv('data/left_only.csv')


Books in Freihand matched with signature: 74.4 %


In [23]:
# Merging on bvb to get more infromation about every document

both.drop('_merge' , axis=1, inplace=True)
merged = pd.merge(both, df_full, left_on='bvb', right_on='id', how= 'left', indicator=True)

merged.drop_duplicates(inplace=True)

In [24]:
len(merged)/len(df_freihand) * 100 

66.7893799460184

In [25]:
text_cols = ['text', 'text_1', 'text_2', 'text_3']
for col in text_cols: 
    merged[col] = merged[col].str.replace(' ', '_')

In [26]:
merged

,bvb,title,year,signature,collection,inventory_nr,inventory_date,signature_clean,sys,text,text_3,text_2,text_1,id,lang,authors,parentId,_merge
0,BV000001769,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only
1,BV001953292,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),BV001953292,ger,"Hauser, Arnold",BV000001769,both
2,BV005486424,Homo caelestis,1965,Mk 1020-5651/1,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only
4,BV038967812,Homo caelestis,1965,Mk 1020-5651/1,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),BV038967812,ger,"Steinen, Wolfram von den",NaN,both
5,BV005486424,Homo caelestis,1965,Mk 1020-5651/2,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244369,BV006647878,Eine Bildhauerwerkstatt des Barock: Die Benede...,1941,Ca-BEN 398-5410,BIB,,0,Ca-BEN 398,Ca-BEN 398,Benedetti,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only
244370,BV008743691,Eine Bildhauerwerkstatt des Barock: Die Benede...,1941,Ca-BEN 398-5410,BIB,,0,Ca-BEN 398,Ca-BEN 398,Benedetti,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only
244371,BV017126597,Die heimische Bildhauerfamilie Rainalter,1937,Ca-RAI 380-5370,BIB,,0,Ca-RAI 380,Ca-RAI 380,Rainalter,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only
244372,BV006647878,Die heimische Bildhauerfamilie Rainalter,1937,Ca-RAI 380-5370,BIB,,0,Ca-RAI 380,Ca-RAI 380,Rainalter,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only


# Acquisition date

In [27]:
# Regular expression pattern to match the year (four digits)

# define a regex pattern to extract the first occurrence of a year number

year_pattern = r"\b(19[0-9]{2}|20[0-2][0-9])\b"

merged['inventory_year'] = merged['inventory_nr'].apply(lambda x: re.search(year_pattern, str(x)).group(0) if re.search(year_pattern, str(x)) else None)


In [28]:
merged

,bvb,title,year,signature,collection,inventory_nr,inventory_date,signature_clean,sys,text,text_3,text_2,text_1,id,lang,authors,parentId,_merge,inventory_year
0,BV000001769,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only,2019
1,BV001953292,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),BV001953292,ger,"Hauser, Arnold",BV000001769,both,2019
2,BV005486424,Homo caelestis,1965,Mk 1020-5651/1,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only,None
4,BV038967812,Homo caelestis,1965,Mk 1020-5651/1,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),BV038967812,ger,"Steinen, Wolfram von den",NaN,both,None
5,BV005486424,Homo caelestis,1965,Mk 1020-5651/2,BIB,,0,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244369,BV006647878,Eine Bildhauerwerkstatt des Barock: Die Benede...,1941,Ca-BEN 398-5410,BIB,,0,Ca-BEN 398,Ca-BEN 398,Benedetti,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only,None
244370,BV008743691,Eine Bildhauerwerkstatt des Barock: Die Benede...,1941,Ca-BEN 398-5410,BIB,,0,Ca-BEN 398,Ca-BEN 398,Benedetti,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only,None
244371,BV017126597,Die heimische Bildhauerfamilie Rainalter,1937,Ca-RAI 380-5370,BIB,,0,Ca-RAI 380,Ca-RAI 380,Rainalter,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only,None
244372,BV006647878,Die heimische Bildhauerfamilie Rainalter,1937,Ca-RAI 380-5370,BIB,,0,Ca-RAI 380,Ca-RAI 380,Rainalter,,Alte_Künstler_(geboren_vor_1870),Italienische_Künstler,NaN,NaN,NaN,NaN,left_only,None


In [89]:
# Export

export_columns = ['bvb', 'title', 'signature', 'inventory_nr','inventory_date', 'inventory_year', 'lang', 'year', 'sys', 'text', 'text_1', 'text_2', 'text_3']

merged[export_columns].to_csv('data/csv/freihand_signatures.csv', index=False)